# Damping forces from models

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
import pandas as pd
from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)
from vessel_manoeuvring_models.symbols import *

In [ ]:
def get_dataset_name(vmm='vmm_linear', regression='force_regression', id=22773):
    jinja = "{{ vmm }}.{{ regression }}.{{ id }}.data_damping_forces"
    template = jinja.replace(r'{{','{').replace(r'}}','}')
    
    return eval(f"f'{template}'")

def get_dataset(vmm='vmm_linear', regression='force_regression', id=22773):
    
    name = get_dataset_name(vmm=vmm, regression=regression, id=id)
    return catalog.load(name)

def get_dataset_name_joined(vmm='vmm_linear', regression='force_regression', id=22773, join_name='joined'):
    jinja = "{{ vmm }}.{{ regression }}.{{ join_name }}.{{ id }}.data_damping_forces"
    template = jinja.replace(r'{{','{').replace(r'}}','}')
    
    return eval(f"f'{template}'")

def get_dataset_joined(vmm='vmm_linear', regression='force_regression', id=22773):
    
    name = get_dataset_name(vmm=vmm, regression=regression, id=id)
    return catalog.load(name)
    
    

In [ ]:
df_VCT = catalog.load("force_regression.data_scaled_resistance_corrected")
df_VCT['item'] = 'VCT'
#data = df_VCT[['fx','fy','mz','u','v','V','r','beta','delta','thrust','test type']].copy()
#data['item'] = 'VCT'
data = pd.DataFrame()


#vmms = ['vmm_martin']
vmms = ['vmm_martins_simple']
#vmms = ['vmm_linear']
model_test_ids = [22773]

for vmm in vmms:
    data_ = get_dataset(vmm=vmm)
    data_['item'] = get_dataset_name(vmm=vmm)
    data = data.append(data_)
    
for vmm in vmms:
    data_ = get_dataset_joined(vmm=vmm, regression='motion_regression')
    data_['item'] = get_dataset_name_joined(vmm=vmm, regression='motion_regression', join_name='joined')
    data = data.append(data_)
    
data.append(df_VCT)

In [ ]:
import plotly.express as px
from vessel_manoeuvring_models.visualization.plot import test_type_xplot

V = data.groupby(by='V')['item'].count().sort_values().index[-1]
data_V = data.groupby(by='V').get_group(V).copy()

data_V["v*r"] = data_V["v"] * data_V["r"]
data_V["beta"] = -np.arctan2(data_V["v"], data_V["u"])

key = 'mz'

for test_type, df in data_V.groupby(by='test type'):
    
    if test_type == 'resistance':
        continue
    
    x = test_type_xplot.get(test_type, 'delta')
    fig = px.line(df.sort_values(by=x), x=x, y=key, color="item", width=1200, height=300, title=test_type)
    fig.show()
    
fig = px.line(data.groupby(by='test type').get_group('resistance').sort_values(by='u'), x='u', y='fx', color="item", width=1200, height=300, title='resistance')
fig.show()

In [ ]:
captive_plot(data, styles=['-','--','o'])

In [ ]:
data.groupby(by=['test type']).get_group('resistance')